| Model	| RPM | TPM | RPD | Batch Enqueued Tokens |
|-------|------|-----|----|--------------------|
| Gemini 3 Pro Preview | 50 | 1,000,000 | 1,000 | 50,000,000 |
|Gemini 2.5 Pro	| 150	|2,000,000	|10,000	|5,000,000 |
|Gemini 2.5 Flash|1,000	|1,000,000	|10,000	|3,000,000|
|Gemini 2.5 Flash Preview|1,000	|1,000,000|10,000|3,000,000|
|Gemini 2.5 Flash-Lite|4,000|4,000,000	|*	|10,000,000|
|Gemini 2.5 Flash-Lite Preview|4,000|4,000,000	|*	|10,000,000|

In [ ]:
import time
import backoff
import threading
import itertools
import pyarrow as pa
import pyarrow.parquet as pq

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

import google.generativeai as genai
from google.generativeai.types import GenerationConfig
import google.api_core.exceptions as gexc

In [ ]:
TEMPLATE_DIR = '../00.data/00.wikidata/03.wikidata_template/'
TEMPLATE_NAME = ['00.original_template_500.parquet',
                 '01.subject_shuffled_template_500.parquet',
                 '02.object_shuffled_template_500.parquet',
                 '03.property_scoped_subject_shuffled_template_500.parquet',
                 '04.property_scoped_object_shuffled_template_500.parquet']

TEMPLATE_LIST = [pq.read_table(TEMPLATE_DIR + name).to_pandas() for name in TEMPLATE_NAME]

In [ ]:
API_KEYS = [
]

In [ ]:
QUOTA = 500
PLAN = {
    'en': (API_KEYS[3], QUOTA),
    'fr': (API_KEYS[4], QUOTA),
    'de': (API_KEYS[5], QUOTA),
    'es': (API_KEYS[6], QUOTA),
    'it': (API_KEYS[3], QUOTA),
    'pt': (API_KEYS[4], QUOTA),
    'ko': (API_KEYS[5], QUOTA),
    'ja': (API_KEYS[6], QUOTA),
}
LANGUAGE_LIST = ['en', 'fr', 'de', 'es', 'it', 'pt', 'ko', 'ja']

In [ ]:
MODEL_ID = "gemini-2.5-flash"
CONFIG = GenerationConfig()
MAX_WORKERS = 10

In [ ]:
@backoff.on_exception(
    backoff.expo,
    (gexc.ResourceExhausted, 
     gexc.InternalServerError, 
     gexc.ServiceUnavailable),
    max_tries=6,
    jitter=backoff.full_jitter)
def safe_generate(model, prompt, cfg):
    return model.generate_content(prompt, generation_config=cfg, request_options={"timeout": 60})

class KeyLimiter:
    def __init__(self, rpm=600):
        self.count = 0
        self.reset_t = time.monotonic() + 60
        self.min_interval = 60.0 / rpm
        self.last_call = 0.0
        self.lock = threading.Lock()

    def wait(self):
        while True:
            with self.lock:
                now = time.monotonic()

                # 분 리셋
                if now >= self.reset_t:
                    self.count = 0
                    self.reset_t = now + 60

                # RPM 제한
                if self.count < 600:
                    # burst 방지
                    wait_time = self.min_interval - (now - self.last_call)
                    if wait_time > 0:
                        pass
                    else:
                        self.count += 1
                        self.last_call = now
                        return
                else:
                    wait_time = self.reset_t - now

            # sleep은 lock 밖에서
            time.sleep(max(wait_time, 0.01))

LIMITER = {key: KeyLimiter() for key in API_KEYS}
tls = threading.local()

def gemini_call(task):
    idx, lang, prompt, key = task

    LIMITER[key].wait()

    if not hasattr(tls, "models"):
        tls.models = {}

    if key not in tls.models:
        genai.configure(api_key=key)
        tls.models[key] = genai.GenerativeModel(MODEL_ID)

    model = tls.models[key]
    r = safe_generate(model, prompt, CONFIG)
    return idx, lang, (r.text or "ERROR")

def process_template(df, template_idx, output_path):
    tasks = []

    for lang, (key, quota) in PLAN.items():
        pcol = f"prompt_{lang}"

        for i in df.index[:quota]:
            tasks.append((i, lang, df.at[i, pcol], key))

    print(f"[Template {template_idx}] 총 요청: {len(tasks)}")

    start = time.time()
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
        futures = [pool.submit(gemini_call, t) for t in tasks]

        for f in tqdm(as_completed(futures), total=len(tasks), desc=f"Gemini T{template_idx}", unit="req"):
            i, lang, txt = f.result()
            df.at[i, f"response_{lang}"] = txt

    print(f"[Template {template_idx}] 완료: {time.time() - start:.1f} sec")

    pq.write_table(pa.Table.from_pandas(df), output_path)
    print("[저장 완료] →", output_path)

    return df


In [ ]:
OUTPUT_BASE = "../00.data/01.model_response/00.gemini_response/"
TEMPLATE_OUTPUT_NAME = [
    "00.original_response_500.parquet",
    "01.subject_shuffled_response_500.parquet",
    "02.object_shuffled_response_500.parquet",
    "03.property_scoped_subject_shuffled_response_500.parquet",
    "04.property_scoped_object_shuffled_response_500.parquet",
]

In [ ]:
for t_idx, df in enumerate(TEMPLATE_LIST):
    out_name = TEMPLATE_OUTPUT_NAME[t_idx]
    output_file = OUTPUT_BASE + out_name
    process_template(df.copy(), t_idx, output_file)

In [ ]:
out_name = TEMPLATE_OUTPUT_NAME[2]
output_file = OUTPUT_BASE + out_name
process_template(TEMPLATE_LIST[2], 2, output_file)
print(2)

In [ ]:
out_name = TEMPLATE_OUTPUT_NAME[3]
output_file = OUTPUT_BASE + out_name
process_template(TEMPLATE_LIST[3], 3, output_file)
print(3)

In [ ]:
out_name = TEMPLATE_OUTPUT_NAME[4]
output_file = OUTPUT_BASE + out_name
process_template(TEMPLATE_LIST[4], 4, output_file)
print(4)

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
tmp = pq.read_table('../00.data/01.model_response/00.gemini_response/04.property_scoped_object_shuffled_response_500.parquet').to_pandas()

In [ ]:
tmp['prompt_en'][0]